In [31]:
"""
A simple example for Reinforcement Learning using table lookup Q-learning method.
An agent "o" is on the left of a 1 dimensional world, the treasure is on the rightmost location.
Run this program and to see how the agent will improve its strategy of finding the treasure.
View more on my tutorial page: https://morvanzhou.github.io/tutorials/
"""

import numpy as np
import pandas as pd
import time

np.random.seed(2)  # reproducible


N_STATES = 6   # the length of the 1 dimensional world
ACTIONS = ['left', 'right']     # available actions
EPSILON = 0.9   # greedy police
ALPHA = 0.1     # learning rate
GAMMA = 0.9    # discount factor
MAX_EPISODES = 13   # maximum episodes
FRESH_TIME = 0.3    # fresh time for one move


def build_q_table(n_states, actions):
    table = pd.DataFrame(
        np.zeros((n_states, len(actions))),     # q_table initial values
        columns=actions,    # actions's name
    )
    # print(table)    # show table
    return table


def choose_action(state, q_table):
    # This is how to choose an action
    state_actions = q_table.iloc[state, :]
    if (np.random.uniform() > EPSILON) or ((state_actions == 0).all()):  # act non-greedy or state-action have no value
        action_name = np.random.choice(ACTIONS)
    else:   # act greedy
        action_name = state_actions.idxmax()    # replace argmax to idxmax as argmax means a different function in newer version of pandas
    return action_name


def get_env_feedback(S, A):
    # This is how agent will interact with the environment
    if A == 'right':    # move right
        if S == N_STATES - 2:   # terminate
            S_ = 'terminal'
            R = 1
        else:
            S_ = S + 1
            R = 0
    else:   # move left
        R = 0
        if S == 0:
            S_ = S  # reach the wall
        else:
            S_ = S - 1
    return S_, R


def update_env(S, episode, step_counter):
    # This is how environment be updated
    env_list = ['-']*(N_STATES-1) + ['T']   # '---------T' our environment
    if S == 'terminal':
        interaction = 'Episode %s: total_steps = %s' % (episode+1, step_counter)
        print('\r{}'.format(interaction), end='')
        time.sleep(2)
        print('\r                                ', end='')
    else:
        env_list[S] = 'o'
        interaction = ''.join(env_list)
        print('\r{}'.format(interaction), end='')
        time.sleep(FRESH_TIME)


def rl():
    # main part of RL loop
    q_table = build_q_table(N_STATES, ACTIONS)
    for episode in range(MAX_EPISODES):
        step_counter = 0
        S = 0
        is_terminated = False
        update_env(S, episode, step_counter)
        while not is_terminated:

            A = choose_action(S, q_table)
            S_, R = get_env_feedback(S, A)  # take action & get next state and reward
            q_predict = q_table.loc[S, A]
            if S_ != 'terminal':
                q_target = R + GAMMA * q_table.iloc[S_, :].max()   # next state is not terminal
            else:
                q_target = R     # next state is terminal
                is_terminated = True    # terminate this episode

            q_table.loc[S, A] += ALPHA * (q_target - q_predict)  # update
            S = S_  # move to next state

            update_env(S, episode, step_counter+1)
            step_counter += 1
    return q_table


if __name__ == "__main__":
    q_table = rl()
    print('\r\nQ-table:\n')
    print(q_table)

In [3]:
pip install gym

In [4]:
import numpy as np
import gym
import random
import time
from IPython.display import clear_output

In [6]:
env=gym.make('FrozenLake-v1')

In [7]:
action_space_size=env.action_space.n
state_space_size=env.observation_space.n

q_table = np.zeros((state_space_size,action_space_size))
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [8]:
num_episodes=1000
max_steps_per_episodes=100

learning_rate=0.1
discount_rate=0.99

exploration_rate=1
max_exploration_rate=1
min_exploration_rate=0.01
exploration_decay_rate=0.001


In [9]:
rewards_all_episodes=[]

for episodes in range(num_episodes):
    state = env.reset()
    
    done = False
    rewards_current_episodes=0
    
    for step in range(max_steps_per_episodes):
        
        exploration_rate_threshold = random.uniform(0,1)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:])
        else:
            action = env.action_space.sample()
            
        new_state,reward,done,info =env.step(action)
        
        
        q_table[state, action]=q_table[state, action]*(1-learning_rate) + learning_rate*(reward+ discount_rate*np.max(q_table[new_state,:]))
        
        
        state =new_state
        rewards_current_episodes += reward
        
        if done == True:
            break
            
    exploration_rate = min_exploration_rate+(max_exploration_rate-min_exploration_rate)*np.exp(-exploration_decay_rate*episodes)
    
    rewards_all_episodes.append(rewards_current_episodes)
    
    
print(q_table)

[[0.23906075 0.20777409 0.22162565 0.21298487]
 [0.07622208 0.13460238 0.08809653 0.16870272]
 [0.13598047 0.08653239 0.06328902 0.09618688]
 [0.0419054  0.0281954  0.01093418 0.01669488]
 [0.24892936 0.14707465 0.12581929 0.16037451]
 [0.         0.         0.         0.        ]
 [0.10665247 0.04594208 0.05457077 0.02813521]
 [0.         0.         0.         0.        ]
 [0.11900425 0.14307522 0.1448356  0.26349353]
 [0.14007101 0.2825155  0.1896174  0.11727365]
 [0.35903219 0.2259131  0.20727104 0.1225501 ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.12283243 0.24671933 0.25069725 0.23561782]
 [0.35205077 0.61676966 0.68473803 0.45536769]
 [0.         0.         0.         0.        ]]


In [11]:
#for episodes in range(10):
state = env.reset()
done=False

time.sleep(1)

for step in range(max_steps_per_episodes):
    clear_output(wait=True)
    env.render()
    time.sleep(0.3)

    action = np.argmax(q_table[state,:])
    new_state,reward,done,info =env.step(action)


    if done:
        clear_output(wait=True)
        env.render()
        if reward ==1:
            print('*******Reached goal***********')
            time.sleep(3)
        else:
            print('***********mission failed**********')
            time.sleep(3)
        clear_output(wait=True)
        break

    state=new_state
        
        
env.close()
        
        

  (Right)
SFFF
FHFH
FFFH
HFFG
*******Reached goal***********
